In [1]:
# tylko nowe cechy
# liczba nowych cech to połowa oryginalnych cech
# sqrt(n) zaokrąglony w góre
# klasyfikatory - zafiksowac sensowne ustawienia
# wagi sumowane do 1
# lecim po wszystkich zbiorach czemu nie
# k-fold cross validation
from multi_imbalance.datasets import load_datasets
import experiment3 as ex
from sklearn.model_selection import StratifiedKFold
from collections import Counter, defaultdict
from multi_imbalance.utils.metrics import gmean_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

/home/dhorna/anaconda3/envs/mgr/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
datasets = load_datasets()

In [3]:
config = ex.config_calculation_strategy2(datasets)

In [4]:
config

{'1czysty-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 24,
   'units_3rd_layer': 16,
   'units_latent_layer': 6},
  'weighted_triplet_loss': True},
 '2delikatne-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 24,
   'units_3rd_layer': 16,
   'units_latent_layer': 6},
  'weighted_triplet_loss': True},
 '3mocniej-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 24,
   'units_3rd_layer': 16,
   'units_latent_layer': 6},
  'weighted_triplet_loss': True},
 '4delikatne-bezover-cut': {'nn_config': {'units_1st_layer': 2,
   'units_2nd_layer': 24,
   'units_3rd_layer': 16,
   'units_latent_layer': 6},
  'weighted_triplet_loss': True},
 'balance-scale': {'nn_config': {'units_1st_layer': 4,
   'units_2nd_layer': 24,
   'units_3rd_layer': 16,
   'units_latent_layer': 6},
  'weighted_triplet_loss': True},
 'cleveland': {'nn_config': {'units_1st_layer': 13,
   'units_2nd_layer': 39,
   'units_3rd_layer': 26,
   'units_latent_layer': 6},
  'weighte

In [5]:
def instantiate_classifiers():
    classifiers = {
            "RF24": RandomForestClassifier(random_state=0, min_samples_leaf=2, min_samples_split=4),
            "RF4": RandomForestClassifier(random_state=0, max_depth=4),
            "RF5": RandomForestClassifier(random_state=0, max_depth=5),
            "RF6": RandomForestClassifier(random_state=0, max_depth=5),
            "RF7": RandomForestClassifier(random_state=0, max_depth=7),
            "KNN3": KNeighborsClassifier(n_neighbors=3),
            "KNN1": KNeighborsClassifier(n_neighbors=3)
        } 
    return classifiers

In [6]:
result_dict = defaultdict(list)

# for dataset_name in list(datasets.keys())[5:6]:
for dataset_name in list(datasets.keys()):
    print(dataset_name)
    result_dict["dataset"].append(dataset_name)
    X, y = datasets[dataset_name]['data'], datasets[dataset_name]['target']
    skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    n_splits = skf.get_n_splits(X, y)
    fold_result_dict = defaultdict(list)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        weights = ex.weights_calculation_strategy1(X_train, y_train)
        nn_config =  config[dataset_name]['nn_config']
        # pretrain ANN triplets
        embeddings_train, embeddings_test = ex.train_triplets(X_train, y_train, X_test, y_test, weights,nn_config)
        X_train_new = np.hstack([X_train, embeddings_train])
        X_test_new = np.hstack([X_test, embeddings_test])
        
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(X_train, y_train)
            preds = clf.predict(X_test)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-old-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-old-features"].append(gmean)
            
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(X_train_new, y_train)
            preds = clf.predict(X_test_new)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-old+new-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-old+new-features"].append(gmean)
            
        classifiers = instantiate_classifiers()
        for clf_name, clf in classifiers.items():
            clf.fit(embeddings_train, y_train)
            preds = clf.predict(embeddings_test)

            gmean = gmean_score(y_test, preds)
            f1 = f1_score(y_test, preds, average='macro')
            fold_result_dict[f"{clf_name}-f1-new-features"].append(f1)
            fold_result_dict[f"{clf_name}-gmean-new-features"].append(gmean)
    for k in fold_result_dict:
        result_dict[k].append(np.mean(fold_result_dict[k]))
        
            
            #f1, gmean
    # mean over folds
    # wiersze: datasety, 
    # kolumny -> klasyfikatory (przed dodaniem cech, po dodaniu, na samych nowych cechach)

1czysty-cut
Train Epoch: 1 [0/960 (0%)]	Loss: 186.123215

Test set: Average loss: 123.2236

Train Epoch: 2 [0/960 (0%)]	Loss: 21.596069

Test set: Average loss: 49.3352

Train Epoch: 3 [0/960 (0%)]	Loss: 11.693846

Test set: Average loss: 31.7832

Train Epoch: 4 [0/960 (0%)]	Loss: 2.828129

Test set: Average loss: 24.4918

Train Epoch: 5 [0/960 (0%)]	Loss: 4.979189

Test set: Average loss: 18.3204

Train Epoch: 6 [0/960 (0%)]	Loss: 2.476213

Test set: Average loss: 15.2276

Train Epoch: 7 [0/960 (0%)]	Loss: 2.110057

Test set: Average loss: 13.8955

Train Epoch: 8 [0/960 (0%)]	Loss: 1.589484

Test set: Average loss: 11.2563

Train Epoch: 9 [0/960 (0%)]	Loss: 2.139846

Test set: Average loss: 10.4078

Train Epoch: 10 [0/960 (0%)]	Loss: 0.939745

Test set: Average loss: 9.1144

Train Epoch: 11 [0/960 (0%)]	Loss: 1.653927

Test set: Average loss: 8.3760

Train Epoch: 12 [0/960 (0%)]	Loss: 0.789339

Test set: Average loss: 8.1154

Train Epoch: 13 [0/960 (0%)]	Loss: 1.427342

Test set: Aver

  0%|          | 0/960 [00:00<?, ?it/s]


Test set: Average loss: 4.3664



100%|██████████| 240/240 [00:00<00:00, 2481.59it/s]


Train Epoch: 1 [0/960 (0%)]	Loss: 142.210388

Test set: Average loss: 81.7019

Train Epoch: 2 [0/960 (0%)]	Loss: 20.397928

Test set: Average loss: 31.0112

Train Epoch: 3 [0/960 (0%)]	Loss: 9.091957

Test set: Average loss: 21.9429

Train Epoch: 4 [0/960 (0%)]	Loss: 4.327725

Test set: Average loss: 13.6749

Train Epoch: 5 [0/960 (0%)]	Loss: 1.813901

Test set: Average loss: 11.1543

Train Epoch: 6 [0/960 (0%)]	Loss: 1.985476

Test set: Average loss: 9.3255

Train Epoch: 7 [0/960 (0%)]	Loss: 1.996491

Test set: Average loss: 7.5569

Train Epoch: 8 [0/960 (0%)]	Loss: 1.636973

Test set: Average loss: 6.7756

Train Epoch: 9 [0/960 (0%)]	Loss: 1.487133

Test set: Average loss: 6.8517

Train Epoch: 10 [0/960 (0%)]	Loss: 1.034161

Test set: Average loss: 5.7079

Train Epoch: 11 [0/960 (0%)]	Loss: 0.703691

Test set: Average loss: 5.2372

Train Epoch: 12 [0/960 (0%)]	Loss: 1.436698

Test set: Average loss: 4.5975

Train Epoch: 13 [0/960 (0%)]	Loss: 0.615601

Test set: Average loss: 4.6527



...

100%|██████████| 43/43 [00:00<00:00, 2651.73it/s]


Train Epoch: 1 [0/172 (0%)]	Loss: 0.102040

Test set: Average loss: 0.9966

Train Epoch: 2 [0/172 (0%)]	Loss: 1.101036

Test set: Average loss: 0.7440

Train Epoch: 3 [0/172 (0%)]	Loss: 0.376117

Test set: Average loss: 0.5844

Train Epoch: 4 [0/172 (0%)]	Loss: 0.375425

Test set: Average loss: 0.4897

Train Epoch: 5 [0/172 (0%)]	Loss: 0.566587

Test set: Average loss: 0.4260

Train Epoch: 6 [0/172 (0%)]	Loss: 0.089709

Test set: Average loss: 0.3771

Train Epoch: 7 [0/172 (0%)]	Loss: 0.143931

Test set: Average loss: 0.3387

Train Epoch: 8 [0/172 (0%)]	Loss: 0.417684

Test set: Average loss: 0.3092

Train Epoch: 9 [0/172 (0%)]	Loss: 0.387803

Test set: Average loss: 0.2870

Train Epoch: 10 [0/172 (0%)]	Loss: 0.086998

Test set: Average loss: 0.2705

Train Epoch: 11 [0/172 (0%)]	Loss: 0.067690

Test set: Average loss: 0.2580

Train Epoch: 12 [0/172 (0%)]	Loss: 0.100156

Test set: Average loss: 0.2470

Train Epoch: 13 [0/172 (0%)]	Loss: 0.325870

Test set: Average loss: 0.2410

Train Ep

  0%|          | 0/43 [00:00<?, ?it/s]


Test set: Average loss: 0.1821



100%|██████████| 43/43 [00:00<00:00, 2935.85it/s]


In [7]:
results_df = pd.DataFrame.from_dict(result_dict)

In [8]:
results_df

,dataset,RF24-f1-old-features,RF24-gmean-old-features,RF4-f1-old-features,RF4-gmean-old-features,RF5-f1-old-features,RF5-gmean-old-features,RF6-f1-old-features,RF6-gmean-old-features,RF7-f1-old-features,...,RF5-f1-new-features,RF5-gmean-new-features,RF6-f1-new-features,RF6-gmean-new-features,RF7-f1-new-features,RF7-gmean-new-features,KNN3-f1-new-features,KNN3-gmean-new-features,KNN1-f1-new-features,KNN1-gmean-new-features
0,1czysty-cut,0.927800,0.918606,0.905429,0.896495,0.905533,0.902969,0.905533,0.902969,0.928631,...,0.889370,0.842573,0.889370,0.842573,0.926697,0.899423,0.925603,0.926270,0.925603,0.926270
1,2delikatne-cut,0.785844,0.751689,0.760193,0.696414,0.777951,0.725794,0.777951,0.725794,0.786406,...,0.712561,0.631482,0.712561,0.631482,0.729454,0.669051,0.694476,0.652755,0.694476,0.652755
2,3mocniej-cut,0.573803,0.463827,0.498805,0.082434,0.578375,0.416180,0.578375,0.416180,0.584513,...,0.495371,0.206993,0.495371,0.206993,0.541174,0.390525,0.502893,0.397908,0.502893,0.397908
3,4delikatne-bezover-cut,0.818686,0.804856,0.829701,0.794818,0.827701,0.815232,0.827701,0.815232,0.833611,...,0.820613,0.790633,0.820613,0.790633,0.814441,0.794822,0.807767,0.795864,0.807767,0.795864
4,balance-scale,0.613201,0.096373,0.602859,0.096128,0.609438,0.096721,0.609438,0.096721,0.608746,...,0.843861,0.840390,0.843861,0.840390,0.842684,0.837166,0.844850,0.837301,0.844850,0.837301
5,cleveland,0.316170,0.083242,0.285711,0.073227,0.284363,0.074887,0.284363,0.074887,0.296950,...,0.254998,0.055094,0.254998,0.055094,0.236465,0.037673,0.208567,0.032355,0.208567,0.032355
6,cleveland_v2,0.296382,0.041099,0.269112,0.025991,0.266816,0.025892,0.266816,0.025892,0.323781,...,0.251239,0.023226,0.251239,0.023226,0.265849,0.024583,0.330762,0.045023,0.330762,0.045023
7,cmc,0.522486,0.507424,0.509892,0.479466,0.514592,0.487395,0.514592,0.487395,0.541080,...,0.502229,0.483675,0.502229,0.483675,0.496102,0.477333,0.452009,0.428192,0.452009,0.428192
8,dermatology,0.963844,0.955977,0.966801,0.958125,0.969889,0.961607,0.969889,0.961607,0.969889,...,0.938894,0.928182,0.938894,0.928182,0.936037,0.925236,0.929513,0.921905,0.929513,0.921905
9,glass,0.704754,0.494085,0.638555,0.294955,0.688237,0.419342,0.688237,0.419342,0.704035,...,0.481610,0.133723,0.481610,0.133723,0.510393,0.165051,0.479263,0.131605,0.479263,0.131605


In [9]:
results_df.filter(regex=("dataset|RF24-f1*"))

,dataset,RF24-f1-old-features,RF24-f1-old+new-features,RF24-f1-new-features
0,1czysty-cut,0.927800,0.952060,0.928708
1,2delikatne-cut,0.785844,0.753503,0.737488
2,3mocniej-cut,0.573803,0.562462,0.530239
3,4delikatne-bezover-cut,0.818686,0.824185,0.814661
4,balance-scale,0.613201,0.868787,0.835306
5,cleveland,0.316170,0.320465,0.253846
6,cleveland_v2,0.296382,0.328450,0.237914
7,cmc,0.522486,0.507252,0.487225
8,dermatology,0.963844,0.954498,0.933005
9,glass,0.704754,0.643610,0.500152


In [10]:
results_df.filter(regex=("dataset|RF24-gmean*"))

,dataset,RF24-gmean-old-features,RF24-gmean-old+new-features,RF24-gmean-new-features
0,1czysty-cut,0.918606,0.931842,0.898706
1,2delikatne-cut,0.751689,0.711400,0.692040
2,3mocniej-cut,0.463827,0.442662,0.399890
3,4delikatne-bezover-cut,0.804856,0.809553,0.792236
4,balance-scale,0.096373,0.862106,0.820406
5,cleveland,0.083242,0.092945,0.040323
6,cleveland_v2,0.041099,0.066348,0.020345
7,cmc,0.507424,0.489697,0.471869
8,dermatology,0.955977,0.945228,0.922429
9,glass,0.494085,0.338161,0.138903


In [11]:
results_df.filter(regex=("dataset|RF4-f1*"))

,dataset,RF4-f1-old-features,RF4-f1-old+new-features,RF4-f1-new-features
0,1czysty-cut,0.905429,0.891745,0.849167
1,2delikatne-cut,0.760193,0.710742,0.669474
2,3mocniej-cut,0.498805,0.484155,0.469984
3,4delikatne-bezover-cut,0.829701,0.818704,0.783402
4,balance-scale,0.602859,0.880509,0.860566
5,cleveland,0.285711,0.311014,0.238202
6,cleveland_v2,0.269112,0.313883,0.240249
7,cmc,0.509892,0.520677,0.505175
8,dermatology,0.966801,0.948217,0.932925
9,glass,0.638555,0.561530,0.475988


In [12]:
results_df.filter(regex=("dataset|RF4-gmean*"))

,dataset,RF4-gmean-old-features,RF4-gmean-old+new-features,RF4-gmean-new-features
0,1czysty-cut,0.896495,0.841461,0.775416
1,2delikatne-cut,0.696414,0.614137,0.548453
2,3mocniej-cut,0.082434,0.116086,0.151661
3,4delikatne-bezover-cut,0.794818,0.768673,0.726555
4,balance-scale,0.096128,0.900217,0.879110
5,cleveland,0.073227,0.050346,0.034525
6,cleveland_v2,0.025991,0.053098,0.020507
7,cmc,0.479466,0.498814,0.486892
8,dermatology,0.958125,0.937600,0.921729
9,glass,0.294955,0.167967,0.127638


In [13]:
results_df.filter(regex=("dataset|RF5-f1*"))

,dataset,RF5-f1-old-features,RF5-f1-old+new-features,RF5-f1-new-features
0,1czysty-cut,0.905533,0.939372,0.889370
1,2delikatne-cut,0.777951,0.761364,0.712561
2,3mocniej-cut,0.578375,0.546150,0.495371
3,4delikatne-bezover-cut,0.827701,0.836061,0.820613
4,balance-scale,0.609438,0.878194,0.843861
5,cleveland,0.284363,0.286149,0.254998
6,cleveland_v2,0.266816,0.327688,0.251239
7,cmc,0.514592,0.524174,0.502229
8,dermatology,0.969889,0.951616,0.938894
9,glass,0.688237,0.597113,0.481610


In [14]:
results_df.filter(regex=("dataset|RF5-gmean*"))

,dataset,RF5-gmean-old-features,RF5-gmean-old+new-features,RF5-gmean-new-features
0,1czysty-cut,0.902969,0.911542,0.842573
1,2delikatne-cut,0.725794,0.704347,0.631482
2,3mocniej-cut,0.416180,0.358702,0.206993
3,4delikatne-bezover-cut,0.815232,0.811168,0.790633
4,balance-scale,0.096721,0.886930,0.840390
5,cleveland,0.074887,0.046743,0.055094
6,cleveland_v2,0.025892,0.062793,0.023226
7,cmc,0.487395,0.501421,0.483675
8,dermatology,0.961607,0.942583,0.928182
9,glass,0.419342,0.250803,0.133723


In [15]:
results_df.filter(regex=("dataset|RF7-f1*"))

,dataset,RF7-f1-old-features,RF7-f1-old+new-features,RF7-f1-new-features
0,1czysty-cut,0.928631,0.949012,0.926697
1,2delikatne-cut,0.786406,0.761774,0.729454
2,3mocniej-cut,0.584513,0.572596,0.541174
3,4delikatne-bezover-cut,0.833611,0.825327,0.814441
4,balance-scale,0.608746,0.890563,0.842684
5,cleveland,0.296950,0.338347,0.236465
6,cleveland_v2,0.323781,0.323488,0.265849
7,cmc,0.541080,0.531153,0.496102
8,dermatology,0.969889,0.954498,0.936037
9,glass,0.704035,0.621217,0.510393


In [16]:
results_df.filter(regex=("dataset|RF7-gmean*"))

,dataset,RF7-gmean-old-features,RF7-gmean-old+new-features,RF7-gmean-new-features
0,1czysty-cut,0.924891,0.930714,0.899423
1,2delikatne-cut,0.747480,0.718708,0.669051
2,3mocniej-cut,0.452692,0.436681,0.390525
3,4delikatne-bezover-cut,0.828137,0.812554,0.794822
4,balance-scale,0.095293,0.909375,0.837166
5,cleveland,0.080578,0.131125,0.037673
6,cleveland_v2,0.053368,0.053433,0.024583
7,cmc,0.523951,0.511828,0.477333
8,dermatology,0.961607,0.945228,0.925236
9,glass,0.489533,0.332356,0.165051


In [17]:
results_df.filter(regex=("dataset|RF6-f1*"))

,dataset,RF6-f1-old-features,RF6-f1-old+new-features,RF6-f1-new-features
0,1czysty-cut,0.905533,0.939372,0.889370
1,2delikatne-cut,0.777951,0.761364,0.712561
2,3mocniej-cut,0.578375,0.546150,0.495371
3,4delikatne-bezover-cut,0.827701,0.836061,0.820613
4,balance-scale,0.609438,0.878194,0.843861
5,cleveland,0.284363,0.286149,0.254998
6,cleveland_v2,0.266816,0.327688,0.251239
7,cmc,0.514592,0.524174,0.502229
8,dermatology,0.969889,0.951616,0.938894
9,glass,0.688237,0.597113,0.481610


In [18]:
results_df.filter(regex=("dataset|RF6-gmean*"))

,dataset,RF6-gmean-old-features,RF6-gmean-old+new-features,RF6-gmean-new-features
0,1czysty-cut,0.902969,0.911542,0.842573
1,2delikatne-cut,0.725794,0.704347,0.631482
2,3mocniej-cut,0.416180,0.358702,0.206993
3,4delikatne-bezover-cut,0.815232,0.811168,0.790633
4,balance-scale,0.096721,0.886930,0.840390
5,cleveland,0.074887,0.046743,0.055094
6,cleveland_v2,0.025892,0.062793,0.023226
7,cmc,0.487395,0.501421,0.483675
8,dermatology,0.961607,0.942583,0.928182
9,glass,0.419342,0.250803,0.133723


In [19]:
results_df.filter(regex=("dataset|KNN1-f1*"))

,dataset,KNN1-f1-old-features,KNN1-f1-old+new-features,KNN1-f1-new-features
0,1czysty-cut,0.950533,0.950533,0.925603
1,2delikatne-cut,0.728790,0.728790,0.694476
2,3mocniej-cut,0.496819,0.496819,0.502893
3,4delikatne-bezover-cut,0.819202,0.819432,0.807767
4,balance-scale,0.575163,0.605228,0.844850
5,cleveland,0.194154,0.194264,0.208567
6,cleveland_v2,0.228293,0.228293,0.330762
7,cmc,0.481749,0.469210,0.452009
8,dermatology,0.866114,0.864971,0.929513
9,glass,0.613173,0.620472,0.479263


In [20]:
results_df.filter(regex=("dataset|KNN1-gmean*"))

,dataset,KNN1-gmean-old-features,KNN1-gmean-old+new-features,KNN1-gmean-new-features
0,1czysty-cut,0.954519,0.954519,0.926270
1,2delikatne-cut,0.688025,0.688025,0.652755
2,3mocniej-cut,0.382030,0.382030,0.397908
3,4delikatne-bezover-cut,0.798464,0.797430,0.795864
4,balance-scale,0.089997,0.094800,0.837301
5,cleveland,0.017399,0.017399,0.032355
6,cleveland_v2,0.010998,0.010998,0.045023
7,cmc,0.459991,0.446469,0.428192
8,dermatology,0.860522,0.857643,0.921905
9,glass,0.434651,0.437128,0.131605


In [21]:
results_df.filter(regex=("dataset|KNN3-f1*"))

,dataset,KNN3-f1-old-features,KNN3-f1-old+new-features,KNN3-f1-new-features
0,1czysty-cut,0.950533,0.950533,0.925603
1,2delikatne-cut,0.728790,0.728790,0.694476
2,3mocniej-cut,0.496819,0.496819,0.502893
3,4delikatne-bezover-cut,0.819202,0.819432,0.807767
4,balance-scale,0.575163,0.605228,0.844850
5,cleveland,0.194154,0.194264,0.208567
6,cleveland_v2,0.228293,0.228293,0.330762
7,cmc,0.481749,0.469210,0.452009
8,dermatology,0.866114,0.864971,0.929513
9,glass,0.613173,0.620472,0.479263


In [22]:
results_df.filter(regex=("dataset|KNN3-gmean*"))

,dataset,KNN3-gmean-old-features,KNN3-gmean-old+new-features,KNN3-gmean-new-features
0,1czysty-cut,0.954519,0.954519,0.926270
1,2delikatne-cut,0.688025,0.688025,0.652755
2,3mocniej-cut,0.382030,0.382030,0.397908
3,4delikatne-bezover-cut,0.798464,0.797430,0.795864
4,balance-scale,0.089997,0.094800,0.837301
5,cleveland,0.017399,0.017399,0.032355
6,cleveland_v2,0.010998,0.010998,0.045023
7,cmc,0.459991,0.446469,0.428192
8,dermatology,0.860522,0.857643,0.921905
9,glass,0.434651,0.437128,0.131605
